In [1]:
import pandas as pd

my_books = pd.read_csv("liked_books.csv", index_col=0)

In [2]:
my_books

,book_id,title,user_id,rating
0,12961964,The Handmaid's Tale,-1,5
1,15769088,Pride and Prejudice,-1,5
2,18887980,The Secret History,-1,5
3,18143977,All the Light We Cannot See,-1,5
4,18902634,The Goldfinch,-1,5
5,2865,Girl with a Pearl Earring,-1,5
6,6457081,Remarkable Creatures,-1,5
7,29983711,Pachinko,-1,5
8,19288043,Gone Girl,-1,5


In [3]:
my_books["book_id"] = my_books["book_id"].astype(str)

In [4]:
csv_book_mapping = {}
with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id

In [5]:
book_set = set(my_books["book_id"])

In [7]:
overlap_users = {}

# overlap_users contains a list of users who liked the books
with open("goodreads_interactions.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.strip().split(",")
        
        book_id = csv_book_mapping.get(csv_id)
        if book_id in book_set:
            if user_id not in overlap_users:
                overlap_users[user_id] = 1
            else:
                overlap_users[user_id] +=1

In [9]:
filtered_users = set([i for i in overlap_users if overlap_users[i] > my_books.shape[0]/5])

In [11]:
interactions = []
with open("goodreads_interactions.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.strip().split(",")
        if user_id in filtered_users:
            book_id = csv_book_mapping[csv_id]
            interactions.append([user_id, book_id, rating])

In [12]:
# creating sparse matrix
interaction = pd.DataFrame(interactions, columns=["user_id", "book_id", "rating"])
interaction = pd.concat([my_books[["user_id", "book_id", "rating"]], interaction])

In [13]:
interaction

,user_id,book_id,rating
0,-1,12961964,5
1,-1,15769088,5
2,-1,18887980,5
3,-1,18143977,5
4,-1,18902634,5
...,...,...,...
33818388,875967,30832670,5
33818389,875967,28443427,5
33818390,875967,11920126,5
33818391,875967,25596982,5


In [14]:
interaction["book_id"] = interaction["book_id"].astype(str)
interaction["user_id"] = interaction["user_id"].astype(str)
interaction["rating"] = pd.to_numeric(interaction["rating"])

In [15]:
interaction["user_id"].unique()

array(['-1', '3', '22', ..., '875556', '875914', '875967'], dtype=object)

In [16]:
interaction["user_index"] = interaction["user_id"].astype("category").cat.codes
# do the same thing for book_index

In [20]:
# use scipy.sparse import coo_matrix
# initialize matrix (can look up documentation) with interaction[rating], [user_index], [book_index]
# convert to csr probably

In [21]:
# find similar users using cosine similarity (in sklearn)
# select most similar using argpartition (?)


In [22]:
# create recommendations & rank them...